<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Ensemble-Methods-on-Diabetes-dataset" data-toc-modified-id="Introduction-to-Ensemble-Methods-on-Diabetes-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Ensemble Methods on Diabetes dataset</a></span><ul class="toc-item"><li><span><a href="#Load-Dataset-Diabetes.csv" data-toc-modified-id="Load-Dataset-Diabetes.csv-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Dataset Diabetes.csv</a></span></li><li><span><a href="#Perform-Normalisation" data-toc-modified-id="Perform-Normalisation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Perform Normalisation</a></span></li><li><span><a href="#Split-The-dataset-ratio-80-and-20" data-toc-modified-id="Split-The-dataset-ratio-80-and-20-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Split The dataset ratio 80 and 20</a></span></li></ul></li><li><span><a href="#Instantinate-all-classifier-want-to-ensemble" data-toc-modified-id="Instantinate-all-classifier-want-to-ensemble-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Instantinate all classifier want to ensemble</a></span><ul class="toc-item"><li><span><a href="#Find-accuracy-all-ensemble-model" data-toc-modified-id="Find-accuracy-all-ensemble-model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Find accuracy all ensemble model</a></span></li></ul></li><li><span><a href="#Intoduction-to-Hard-Voting-and-Soft-Voting" data-toc-modified-id="Intoduction-to-Hard-Voting-and-Soft-Voting-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Intoduction to Hard Voting and Soft Voting</a></span><ul class="toc-item"><li><span><a href="#Pre-requiste:-pip-install-mlxtend" data-toc-modified-id="Pre-requiste:-pip-install-mlxtend-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Pre-requiste: pip install mlxtend</a></span></li><li><span><a href="#Instantinate-EnsembleVoteClassifier" data-toc-modified-id="Instantinate-EnsembleVoteClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Instantinate EnsembleVoteClassifier</a></span></li><li><span><a href="#Find-accuracy-through-Hard-and-Soft-Voting-mechanism" data-toc-modified-id="Find-accuracy-through-Hard-and-Soft-Voting-mechanism-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Find accuracy through Hard and Soft Voting mechanism</a></span></li></ul></li></ul></div>

### Introduction to Ensemble Methods on Diabetes dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,cross_val_predict,cross_val_score

####  Load Dataset Diabetes.csv

In [2]:
df = pd.read_csv(r'Diabetes.csv')

In [3]:
X = df.iloc[:,:8]
y=df['class']

#### Perform Normalisation 

In [4]:
X_std = StandardScaler()
X  = X_std.fit_transform(X)

#### Split The dataset ratio 80 and 20

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=201)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


### Instantinate all classifier want to ensemble

In [7]:
lr = LogisticRegression(random_state=2017)
rf = RandomForestClassifier(random_state=2017,n_estimators=100)
dt = DecisionTreeClassifier(random_state=2017)
bc = BaggingClassifier(n_estimators=100,random_state=2017)
knc = KNeighborsClassifier()
gr = GradientBoostingClassifier(n_estimators=100)

#### Find accuracy all ensemble model

In [8]:
clfs=[]
for clf, labels in zip([lr,rf,dt,bc,knc,gr],['logistic Regression','Random Forest classifier',
                                           'Decision Tree Classifier','Bagging Classifier',
                                           'K Nearest Neigbour Classifier','Gradinet Boosting']):
    
    scores = cross_val_score(clf,X_train,y_train,cv=5)
    print('label: %s  Accuracy: %0.2f '%(labels,np.mean(scores)*100))
    md = clf.fit(X,y)
    clfs.append(md)
    print('label: %s  Test Accuracy: %0.2f '%(labels, accuracy_score(clf.predict(X_test),y_test)*100))
    print('*'*30)


label: logistic Regression  Accuracy: 76.55 
label: logistic Regression  Test Accuracy: 79.22 
******************************
label: Random Forest classifier  Accuracy: 78.50 
label: Random Forest classifier  Test Accuracy: 100.00 
******************************
label: Decision Tree Classifier  Accuracy: 71.66 
label: Decision Tree Classifier  Test Accuracy: 100.00 
******************************
label: Bagging Classifier  Accuracy: 77.86 
label: Bagging Classifier  Test Accuracy: 100.00 
******************************
label: K Nearest Neigbour Classifier  Accuracy: 75.74 
label: K Nearest Neigbour Classifier  Test Accuracy: 79.22 
******************************
label: Gradinet Boosting  Accuracy: 77.04 
label: Gradinet Boosting  Test Accuracy: 88.96 
******************************


### Intoduction to Hard Voting and Soft Voting

#### Pre-requiste: pip install mlxtend

In [12]:
from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier,EnsembleVoteClassifier

#### Instantinate EnsembleVoteClassifier

In [13]:
clfs=[]
ECH = EnsembleVoteClassifier(clfs=[lr,rf,gr],voting='hard')
ECS = EnsembleVoteClassifier(clfs=[lr,rf,gr],voting='soft')

#### Find accuracy through Hard and Soft Voting mechanism 

In [22]:
for clf,label in zip([ECH,ECS],['Ensemble Hard Voting','Ensemble Soft Voting']):
    scores = cross_val_score(clf,X_train,y_train,cv=5,scoring='accuracy')
    print('Label: %s, Training Accuracy %0.2f: '%(label,(np.mean(scores))))
    md = clf.fit(X_train, y_train)
    clfs.append(md)
    print('Label: %s, Testing Accuracy %0.2f: '%(label, accuracy_score(y_test, clf.predict(X_test))))
    print('*'*40)

Label: Ensemble Hard Voting, Training Accuracy 0.79: 
Label: Ensemble Hard Voting, Testing Accuracy 0.77: 
****************************************
Label: Ensemble Soft Voting, Training Accuracy 0.79: 
Label: Ensemble Soft Voting, Testing Accuracy 0.75: 
****************************************
